Extraction de données des livres les plus vendus en 2022 sur amazon

In [80]:
#Importer BeautifulSoup
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
import pandas as pd
from pandas import DataFrame
from lxml import etree

In [24]:
#On copie juste la page 2 du site, car je veux juste les 2 1ères pages 
num = 1    #On initialise à 1

base_url = 'https://www.amazon.fr/s?k=livre+les+plus+vendus+2022&i=stripbooks&page='
page_num = '1'
url_separator = '&__mk_fr_FR=ÅMÅŽÕÑ&crid=34M35DTOHZ523&qid=1673727023&sprefix=livres+les+plus+vendus+2022%2Cstripbooks%2C58&ref=sr_pg_1'

#'https://www.amazon.fr/s?k=livre+les+plus+vendus+2022&i=stripbooks&page=2&__mk_fr_FR=ÅMÅŽÕÑ&crid=34M35DTOHZ523&qid=1673727023&sprefix=livres+les+plus+vendus+2022%2Cstripbooks%2C58&ref=sr_pg_2'

In [25]:
#Headers for requests ( généralement utilisé pour que le site d'amazone authentifie la machine)
Headers = ({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7"})

In [26]:
#On fait appel à l'url et on récupère les données contenues sur les pages
url = base_url + page_num + url_separator

response = requests.get(url, headers = Headers)
print(response)

<Response [200]>


In [27]:
#On récupère les données sous leur format html
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="fr-fr"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<!-- sp:end-feature:cs-optimization -->
<!-- sp:feature:csm:head-open-part2 -->
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=func

In [34]:
#On cherche tous les liens contenus sur la page web
liens = soup.find_all("a", attrs = {"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})

liens

[<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/Face-%C3%A0-nuit-Peter-Robinson/dp/2253000396/ref=sr_1_1?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&amp;crid=34M35DTOHZ523&amp;keywords=livre+les+plus+vendu+2022&amp;qid=1673986755&amp;s=books&amp;sprefix=livres+les+plus+vendus+2022%2Cstripbooks%2C58&amp;sr=1-1"><span class="a-size-medium a-color-base a-text-normal">Face à la nuit</span> </a>,
 <a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/sspa/click?ie=UTF8&amp;spc=MTo1OTUzMzkwNjcxMTY0NTU3OjE2NzM5ODY3NTU6c3BfYXRmOjIwMDE5NjgzMzQyNDk4OjowOjo&amp;url=%2Fvie-en-plus-joli%2Fdp%2FB09MYVNM9W%2Fref%3Dsr_1_2_sspa%3F__mk_fr_FR%3D%25C3%2585M%25C3%2585%25C5%25BD%25C3%2595%25C3%2591%26crid%3D34M35DTOHZ523%26keywords%3Dlivre%2Bles%2Bplus%2Bvendu%2B2022%26qid%3D1673986755%26s%3Dbooks%26sprefix%3Dlivres%2Bles%2Bplus%2Bvendus%2B2022%252Cstripbooks%252C58%26sr%3D1-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hd

In [36]:
#On essaie d'obtenir le premier lien parmi ceux extraient plus haut
lien = liens[0].get('href')

'/Face-%C3%A0-nuit-Peter-Robinson/dp/2253000396/ref=sr_1_1?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=34M35DTOHZ523&keywords=livre+les+plus+vendu+2022&qid=1673986755&s=books&sprefix=livres+les+plus+vendus+2022%2Cstripbooks%2C58&sr=1-1'

In [43]:
#Ici on accède à la page qui contient toute les informations du premier livre
livre_1 = 'https://www.amazon.fr'+lien
livre_1

'https://www.amazon.fr/Face-%C3%A0-nuit-Peter-Robinson/dp/2253000396/ref=sr_1_1?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=34M35DTOHZ523&keywords=livre+les+plus+vendu+2022&qid=1673986755&s=books&sprefix=livres+les+plus+vendus+2022%2Cstripbooks%2C58&sr=1-1'

In [81]:
#On veut extrait le titre du premier livre de cette nouvelle page
nouveau_url = 'https://www.amazon.fr/Face-%C3%A0-nuit-Peter-Robinson/dp/2253000396/ref=sr_1_1?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=34M35DTOHZ523&keywords=livre+les+plus+vendu+2022&qid=1673986755&s=books&sprefix=livres+les+plus+vendus+2022%2Cstripbooks%2C58&sr=1-1'

page_web = requests.get(livre_1, headers = Headers)
print(page_web)

nouveau_soup = BeautifulSoup(page_web.content, 'html.parser')
nouveau_soup

titre_livre_1 = nouveau_soup.find("span", attrs = {"id":"productTitle"}).get_text()
isbn_livre = nouveau_soup.find("span",attrs={"css":"#detailBullets_feature_div>ul>li:nth-child(5)>span>span:nth-child(2)"})
titre_livre_1.strip()
isbn_livre.strip()

<Response [200]>


TypeError: 'NoneType' object is not callable

In [66]:
#On extrait le prix du premier livre
prix_livre_1 = nouveau_soup.find("span", attrs = {"class":"a-size-base a-color-price a-color-price"}).get_text()
prix_livre_1.strip()[:4]    #Le strip enlève les espaces et le slice des 4 derniers krktères supprime les valeurs 
                                #qui apparraissaient avec le prix

'8,90'

In [68]:
#On extrait le nom de l'auteur
nom_auteur_livre_1 = nouveau_soup.find("a", {"class":"a-size-base a-link-normal a-text-normal"}).get_text()
nom_auteur_livre_1

'Peter Robinson'

In [77]:
#Maison d'édition et code ISBN
edition_livre_1 = nouveau_soup.find("span", {"id":"productSubtitle"}).get_text()
edition_livre_1.strip()

isbn_livre_1 = nouveau_soup.find(
print(isbn_livre_1)

None
